# get .npy

In [ ]:
import os
import pickle

import librosa
import numpy as np

from tqdm.auto import tqdm

In [ ]:
class Loader:
    """Loader is responsible for loading an audio file."""

    def __init__(self, sample_rate, duration, mono):
        self.sample_rate = sample_rate
        self.duration = duration
        self.mono = mono

    def load(self, file_path):
        signal = librosa.load(file_path,
                              sr=self.sample_rate,
                              duration=self.duration,
                              mono=self.mono)[0]
        return signal

In [ ]:
class Padder:
    """Padder is responsible to apply padding to an array."""

    def __init__(self, mode="constant"):
        self.mode = mode

    def left_pad(self, array, num_missing_items):
        padded_array = np.pad(array,
                              (num_missing_items, 0),
                              mode=self.mode)
        return padded_array

    def right_pad(self, array, num_missing_items):
        padded_array = np.pad(array,
                              (0, num_missing_items),
                              mode=self.mode)
        return padded_array

In [ ]:
class LogSpectrogramExtractor:
    """LogSpectrogramExtractor extracts log spectrograms (in dB) from a
    time-series signal.
    """

    def __init__(self, frame_size, hop_length):
        self.frame_size = frame_size
        self.hop_length = hop_length

    def extract(self, signal):
        stft = librosa.stft(signal,
                            n_fft=self.frame_size,
                            hop_length=self.hop_length)[:-1]
        spectrogram = np.abs(stft)
        log_spectrogram = librosa.amplitude_to_db(spectrogram)
        return log_spectrogram

In [ ]:
class MinMaxNormaliser:
    """MinMaxNormaliser applies min max normalisation to an array."""

    def __init__(self, min_val, max_val):
        self.min = min_val
        self.max = max_val

    def normalise(self, array):
        norm_array = (array - array.min()) / (array.max() - array.min())
        norm_array = norm_array * (self.max - self.min) + self.min
        return norm_array

    def denormalise(self, norm_array, original_min, original_max):
        array = (norm_array - self.min) / (self.max - self.min)
        array = array * (original_max - original_min) + original_min
        return array

In [ ]:
class Saver:
    """saver is responsible to save features, and the min max values."""

    def __init__(self, feature_save_dir, min_max_values_save_dir):
        self.feature_save_dir = feature_save_dir
        self.min_max_values_save_dir = min_max_values_save_dir

    def save_feature(self, feature, file_path):
        save_path = self._generate_save_path(file_path)
        np.save(save_path, feature)
        return save_path

    def save_min_max_values(self, min_max_values):
        save_path = os.path.join(self.min_max_values_save_dir,
                                 "min_max_values.pkl")
        self._save(min_max_values, save_path)

    @staticmethod
    def _save(data, save_path):
        with open(save_path, "wb") as f:
            pickle.dump(data, f)

    def _generate_save_path(self, file_path):
        file_name = os.path.split(file_path)[1]
        save_path = os.path.join(self.feature_save_dir, file_name + ".npy")
        return save_path


In [ ]:
class PreprocessingPipeline:
    """PreprocessingPipeline processes audio files in a directory, applying
    the following steps to each file:
        1- load a file
        2- pad the signal (if necessary)
        3- extracting log spectrogram from signal
        4- normalise spectrogram
        5- save the normalised spectrogram

    Storing the min max values for all the log spectrograms.
    """

    def __init__(self):
        self.padder = None
        self.extractor = None
        self.normaliser = None
        self.saver = None
        self.min_max_values = {}
        self._loader = None
        self._num_expected_samples = None

    @property
    def loader(self):
        return self._loader

    @loader.setter
    def loader(self, loader):
        self._loader = loader
        self._num_expected_samples = int(loader.sample_rate * loader.duration)

    def process(self, audio_files_dir):
        for root, _, files in os.walk(audio_files_dir):
            for file in tqdm(files):
                file_path = os.path.join(root, file)
                self._process_file(file_path)
                # print(f"Processed file {file_path}")
        self.saver.save_min_max_values(self.min_max_values)

    def _process_file(self, file_path):
        signal = self.loader.load(file_path)
        if self._is_padding_necessary(signal):
            signal = self._apply_padding(signal)
        feature = self.extractor.extract(signal)
        norm_feature = self.normaliser.normalise(feature)
        save_path = self.saver.save_feature(norm_feature, file_path)
        self._store_min_max_value(save_path, feature.min(), feature.max())

    def _is_padding_necessary(self, signal):
        if len(signal) < self._num_expected_samples:
            return True
        return False

    def _apply_padding(self, signal):
        num_missing_samples = self._num_expected_samples - len(signal)
        padded_signal = self.padder.right_pad(signal, num_missing_samples)
        return padded_signal

    def _store_min_max_value(self, save_path, min_val, max_val):
        self.min_max_values[save_path] = {
            "min": min_val,
            "max": max_val
        }

In [ ]:
# preprocessing
FRAME_SIZE = 512
HOP_LENGTH = 256
DURATION = 15.04 # magic number! 1292 -> 1296
SAMPLE_RATE = 22050
MONO = True

ROOT = ""
SPECTROGRAMS_SAVE_DIR = os.path.join(ROOT, "spectrograms")
MIN_MAX_VALUES_SAVE_DIR = os.path.join(ROOT, "MinMaxValue")
FILES_DIR = os.path.join(ROOT, "data")
# instantiate all objectsslice_29_05_2021
loader = Loader(SAMPLE_RATE, DURATION, MONO)
padder = Padder()
log_spectrogram_extractor = LogSpectrogramExtractor(FRAME_SIZE, HOP_LENGTH)
min_max_normaliser = MinMaxNormaliser(0, 1)
saver = Saver(SPECTROGRAMS_SAVE_DIR, MIN_MAX_VALUES_SAVE_DIR)

preprocessing_pipeline = PreprocessingPipeline()
preprocessing_pipeline.loader = loader
preprocessing_pipeline.padder = padder
preprocessing_pipeline.extractor = log_spectrogram_extractor
preprocessing_pipeline.normaliser = min_max_normaliser
preprocessing_pipeline.saver = saver

preprocessing_pipeline.process(FILES_DIR)

# get tfrec

In [ ]:
import tensorflow as tf

In [ ]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _int64_array_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def _float32_array_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [ ]:
def npy_preprocess(root,npy_path):
  file_path = os.path.join(root, npy_path)
  spectrogram = np.load(file_path)
  frequencies, time_frame = spectrogram.shape
  spectrogram = spectrogram.flatten()
  record = tf.train.Example(features=tf.train.Features(feature={
        'frequency': _int64_feature(frequencies),
        'time': _int64_feature(time_frame),
        'spectrograms':  _float32_array_feature(spectrogram),
       
    }))

  return record

In [ ]:
from tqdm.auto import tqdm

In [ ]:
def createTFRecord(tfrec_dir, npy_dir):
    os.makedirs(tfrec_dir, exist_ok=True)

    train_files = os.listdir(npy_dir)
    with tf.io.TFRecordWriter(tfrec_dir , "train.tfrecord") as writer:
        for npy_path in tqdm(train_files):
            record = npy_preprocess(npy_dir,npy_path)
            writer.write(record.SerializeToString())

In [ ]:
createTFRecord(os.path.join(ROOT, "tfrec"), SPECTROGRAMS_SAVE_DIR)

# loading validation

In [ ]:
def parse_function(example_proto):
    features = {
        'frequency': tf.io.FixedLenFeature([], tf.int64),
        'time': tf.io.FixedLenFeature([], tf.int64),
        'spectrograms': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True)
    }
    parsed_features = tf.io.parse_single_example(example_proto, features)

    spectrogram = tf.reshape(parsed_features["spectrograms"],
                        [parsed_features["frequency"], parsed_features['time']])
    spectrogram = tf.transpose(spectrogram)
    return spectrogram

In [ ]:
def get_train_data(tfrec_path):
    train_data = tf.data.TFRecordDataset(tfrec_path)\
        .shuffle(300)\
        .map(parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
        .batch(16)\
        .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
        

    return train_data

In [ ]:
tfrec_path = os.path.join(ROOT, "tfrec/train.tfrecord")
train = get_train_data(tfrec_path)

In [ ]:
for element in train:
  if element.shape[1:] != (1296, 256):
    print(element.shape)
  else:
    pass
print("done!")